# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3eb8d97040>
├── 'a' --> tensor([[-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060]])
└── 'x' --> <FastTreeValue 0x7f3eb8d972e0>
    └── 'c' --> tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                        [-0.2669, -1.9609,  1.0529,  1.8265],
                        [ 0.5557,  0.7122,  0.6872, -0.3841]])

In [4]:
t.a

tensor([[-0.4477, -0.4466, -0.0566],
        [-0.1362,  1.8692, -0.5060]])

In [5]:
%timeit t.a

65.2 ns ± 0.146 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3eb8d97040>
├── 'a' --> tensor([[ 1.4876, -1.3956, -1.7041],
│                   [ 0.5601, -0.0877,  0.2837]])
└── 'x' --> <FastTreeValue 0x7f3eb8d972e0>
    └── 'c' --> tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                        [-0.2669, -1.9609,  1.0529,  1.8265],
                        [ 0.5557,  0.7122,  0.6872, -0.3841]])

In [7]:
%timeit t.a = new_value

60.9 ns ± 0.0774 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060]]),
    x: Batch(
           c: tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                      [-0.2669, -1.9609,  1.0529,  1.8265],
                      [ 0.5557,  0.7122,  0.6872, -0.3841]]),
       ),
)

In [10]:
b.a

tensor([[-0.4477, -0.4466, -0.0566],
        [-0.1362,  1.8692, -0.5060]])

In [11]:
%timeit b.a

53 ns ± 0.0406 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3343,  0.9800, -1.4121],
               [ 0.5790,  2.0409, -0.4248]]),
    x: Batch(
           c: tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                      [-0.2669, -1.9609,  1.0529,  1.8265],
                      [ 0.5557,  0.7122,  0.6872, -0.3841]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.151 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

854 ns ± 0.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

136 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3e0ea9af70>
├── 'a' --> tensor([[[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]],
│           
│                   [[-0.4477, -0.4466, -0.0566],
│                    [-0.1362,  1.8692, -0.5060]]])
└── 'x' --> <FastTreeValue 0x7f3e0f027c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 44.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3eb8d97460>
├── 'a' --> tensor([[-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060],
│                   [-0.4477, -0.4466, -0.0566],
│                   [-0.1362,  1.8692, -0.5060]])
└── 'x' --> <FastTreeValue 0x7f3e0eb14c70>
    └── 'c' --> tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                        [-0.2669, -1.9609,  1.0529,  1.8265],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 72 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]],
       
               [[-0.4477, -0.4466, -0.0566],
                [-0.1362,  1.8692, -0.5060]]]),
    x: Batch(
           c: tensor([[[ 1.5679,  0.8315, -0.3613, -0.2905],
                       [-0.2669, -1.9609,  1.0529,  1.8265],
                       [ 0.5557,  0.7122,  0.6872, -0.3841]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060],
               [-0.4477, -0.4466, -0.0566],
               [-0.1362,  1.8692, -0.5060]]),
    x: Batch(
           c: tensor([[ 1.5679,  0.8315, -0.3613, -0.2905],
                      [-0.2669, -1.9609,  1.0529,  1.8265],
                      [ 0.5557,  0.7122,  0.6872, -0.3841],
                      [ 1.5679,  0.8315, -0.3613, -0.2905],
                      [-0.2669, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
